In [1]:
import dotenv
dotenv.load_dotenv('.env', override=True)

True

In [39]:
import os
from neo4j import GraphDatabase

driver = GraphDatabase.driver(
            os.environ['NEO4J_URI'], 
            auth=(os.environ['NEO4J_USERNAME'], os.environ['NEO4J_PASSWORD'],))

In [58]:
import csv
from tqdm import tqdm

def load_articles(csv_file, start_idx=0, end_idx=None):
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file, delimiter=';')
        with driver.session() as session:
            for i, row in enumerate(tqdm(reader, desc="Loading Articles", unit="article")):
                if i < start_idx:
                    continue

                if end_idx is not None and i >= end_idx:
                    break
                
                session.write_transaction(create_article, row)

def create_article(tx, row):
    title = row['title'].strip()
    abstract = row['abstract'].strip()
    date_publication = row['date_publication']
    keywords = row['keywords'].split(',')

    # Merge the article node
    tx.run("""
    CREATE (a:Article {name: $title})
    SET a.abstract = $abstract, a.date_publication = $date_publication
    WITH a
    UNWIND $keywords AS keyword
    MERGE (b:Keyword {text: keyword})
    MERGE (b)<-[:CONTAIN]-(a)
    """, title=title, abstract=abstract, date_publication=date_publication, keywords=keywords)

In [64]:
load_articles('./data.csv', start_idx=0, end_idx=50)

Loading Articles: 0article [00:00, ?article/s]C:\Users\Lenovo E585 (9-3)\AppData\Local\Temp\ipykernel_8212\2343436497.py:15: DeprecationWarning: write_transaction has been renamed to execute_write
  session.write_transaction(create_article, row)
Loading Articles: 50article [00:06,  8.22article/s]
